In [ ]:
from bs4 import BeautifulSoup
import os
import requests
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from skimage import io
import skimage
import os
from skimage import data
from skimage.color import rgb2hsv
import pandas as pd

# Scraping Beksinski's Works

Fairly self explanatory, searching for Zdislav Beksinski's artworks from Wikiart and downloading them.

In [33]:
DATA_DIR = '/data/'
ARTIST_URL = 'https://www.wikiart.org/en/{artist}/all-works/text-list'
PAINTING_URL = 'https://www.wikiart.org{painting_path}'

if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

In [34]:
artist_name = 'zdislav-beksinski'
url_query = ARTIST_URL.format(artist=artist_name)
artist_page = requests.get(url_query)

In [35]:
try:
    artist_page.raise_for_status()
except requests.exceptions.HTTPError as e:
    print("Error trying to retrieve {}".format(artist_page.url))
    raise e

In [36]:
soup = BeautifulSoup(artist_page.text, 'lxml')
IMAGE_DIR = os.path.join(DATA_DIR, artist_name)
if not os.path.exists(IMAGE_DIR):
    os.makedirs(IMAGE_DIR)

In [37]:
painting_paths = []

for li in soup.find_all('li', {'class': 'painting-list-text-row'}):

    for link in li.find_all('a'):
        href = link.get('href')
        # store in dictionary
        painting_paths.append(href)

print(len(painting_paths))

708


In [38]:
def download_and_save(painting_url):
    r_painting_page = requests.get(painting_url)
    soup = BeautifulSoup(r_painting_page.text, 'lxml')
    for img in soup.find_all('img', {'class': 'ms-zoom-cursor'}):
        img_url = img['src']
        img_url = img_url.split('!')[0]
        filename = img_url.split('/')[-1]

        outfile = os.path.join(IMAGE_DIR, filename)                       
        if not os.path.exists(outfile):                        
            print("downloading {}: {}".format(filename, img_url))
            r = requests.get(img_url, outfile)
            with open(outfile, 'wb') as f:
                f.write(r.content)
        else:
            #print("skipping {}".format(filename))
            pass

In [39]:
#choosing the most interesting to me, Beksinski has over 700 works, so I didn't want to just download all of them.
for path in painting_paths[240:320]:
    painting_path = PAINTING_URL.format(painting_path=path)
    download_and_save(painting_path)

KeyboardInterrupt: 